### Overview

This program will align 2 DNA sequences

I will present the Dynamic Programming algorithm that will be implemented.

It will utilize the following scoring parameters:
    1. Match Premium = 1
    2. Mismatch Penalty = -1
    3. Gap Penalty = -1
    
Test by aligning the following 2 sequences:
S1 = ATCTGTTCTT
S2 = CTTTATGTT

I will also test more broadly (these results will be included in the report)

Include in directory:
    1. Input files
    2. PDF Report:
        a. Overall Description
        b. DP table for S1 and S2 (pathway table)
        c. Resulting alignment.


